In [2]:
import pandas as pd

Technical stuff
- Fill NaNs with (group) averages or previous obs

More data
- geopoints
- Trade statistics

### Load and pre-process WB data

In [25]:
# Load
wb = pd.read_csv("nasdaq/WB_DATA_20240523.csv")

# Pre-process
wb = wb[wb.year.notnull()]
wb['year'] = wb['year'].astype(int)
wb = wb.rename(columns={"series_id": "indicator"})
wb['country_name'] = wb['country_name'].replace('Czechia', 'Czech Republic')
wb['country_name'] = wb['country_name'].replace("Korea, Dem. Peopleâ\x80\x99s Rep.", "Korea, Dem. People's Rep.")

# Pivot
wb = wb.pivot_table(index=['country_code', 'country_name', 'year'], columns='indicator', values='value').reset_index()

### Drop country groups
- These could potentially be used for filling NaNs of group memebers
- Does not have lower coverage than WB countries on average

In [36]:
country_groups = [
    '(UN) Africa', '(UN) Asia', '(UN) Europe', '(UN) Latin America and the Caribbean', 
    '(UN) North America', '(SDG) Central/Southern Asia', '(SDG) Eastern/South Eastern Asia', 
    '(SDG) Europe', '(SDG) Latin America & the Caribbean', '(SDG) Northern America', 
    '(SDG) Sub-Saharan Africa', '(SDG) Western Asia/Northern Africa',
    'Africa Eastern and Southern', 'Africa Western and Central',
    'Association of Southeast Asian Nations (ASEAN-5)',
    'Central Europe and the Baltics', 'Caribbean small states',
    'East Asia & Pacific (excluding high income)', 'East Asia & Pacific',
    'Europe & Central Asia (excluding high income)', 'Europe & Central Asia',
    'Emerging and Developing Europe', 'Euro area',
    'Europe & Central Asia (IDA & IBRD countries)',
    'Latin America & Caribbean (excluding high income)', 'Latin America & Caribbean',
    'Least developed countries: UN classification', 'Middle East & North Africa',
    'Middle East & Central Asia', 'Pacific island small states',
    'South Asia (IDA & IBRD)', 'Small states',
    'Sub-Saharan Africa (excluding high income)',
    'Sub-Saharan Africa (IDA & IBRD countries)',
    'Upper middle income', 'IBRD only', 'IDA & IBRD total', 'IDA total', 'IDA blend',
    'OECD members', 'OECD Member Countries',
    'Other Advanced Economies', 'Other small states',
    'Heavily indebted poor countries (HIPC)', 'High income', 'Low income', 'Lower middle income',
    'Low & middle income', 'Post-demographic dividend', 'Pre-demographic dividend',
    'Fragile and conflict affected situations', 'East Asia & Pacific (IDA & IBRD countries)',
    'Latin America & the Caribbean (IDA & IBRD countries)',
    'Middle East & North Africa (IDA & IBRD countries)', '(WHO) Africa Region',
    '(WHO) America Region', '(WHO) South-East Asia Region',
    '(WHO) Eastern Mediterranean Region', '(WHO) European Region',
    '(WHO) Western Pacific Region', 'Unknown or unspecified', 'Arab World',
    'Developed Asia', 'Early-demographic dividend', 'FAO (Food and Agriculture Organization)',
    'IDA only', 'Late-demographic dividend', 'Middle income',
    'Middle East & North Africa (excluding high income)',
    'World Bank Group', 'Western Europe', 'World',
    'European Union', 'Euro Area', 'Sub-Saharan Africa', 'North America',
    'South Asia']

wb = wb[~wb['country_name'].isin(country_groups)].reset_index(drop=True)

### Drop NaN columns

In [37]:
# Drop cols with more than 80 percent NaNs
threshold = 0.2 * len(wb)
df = wb.dropna(axis=1, thresh=threshold)
print("We have dropped", len(wb.columns)-len(df.columns), "columns with more than 80 percent NaN observations.")

We have dropped 531 columns with more than 80 percent NaN observations.


In [35]:
(df.notnull().sum()/len(df)).mean()

0.4811117081231797